In [ ]:
#importing libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, recall_score, precision_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [ ]:
#loading the dataset 
df = pd.read_csv('/content/drive/MyDrive/fake_job_postings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15185 non-null  object
 8   benefits             10670 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

In [ ]:
df = df[df['location'].notna()]

In [ ]:
df.info()
le = LabelEncoder()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17534 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17534 non-null  int64 
 1   title                17534 non-null  object
 2   location             17534 non-null  object
 3   department           6282 non-null   object
 4   salary_range         2846 non-null   object
 5   company_profile      14288 non-null  object
 6   description          17533 non-null  object
 7   requirements         15015 non-null  object
 8   benefits             10574 non-null  object
 9   telecommuting        17534 non-null  int64 
 10  has_company_logo     17534 non-null  int64 
 11  has_questions        17534 non-null  int64 
 12  employment_type      14277 non-null  object
 13  required_experience  10737 non-null  object
 14  required_education   9713 non-null   object
 15  industry             12879 non-null  object
 16  func

In [ ]:
columns_to_impute = [  'employment_type', 'required_experience', 'industry', 'function']

In [ ]:
#drop unwanted columns 
columns_to_drop = ['description', 'requirements', 'benefits', 'company_profile', 'location', 'salary_range', 'job_id', 'required_education', 'department']
df_dropped = df.drop(columns_to_drop, axis = 1 )

In [ ]:
for column in columns_to_impute:
  print(column)

  df_dropped[column].loc[df_dropped[column].notna(), :] =  le.fit_transform(df_dropped[column].notna())

  mean_value = df_dropped[column].notna().mean()
  df_dropped.fillna(value=mean_value, inplace=True)

employment_type


/usr/local/lib/python3.7/dist-packages/pandas/core/common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


TypeError: ignored

In [ ]:

df_dropped.head()

,title,telecommuting,has_company_logo,has_questions,employment_type,required_experience,industry,function,fraudulent
0,Marketing Intern,0,1,0,Other,Internship,NaN,Marketing,0
1,Customer Service - Cloud Video Production,0,1,0,Full-time,Not Applicable,Marketing and Advertising,Customer Service,0
2,Commissioning Machinery Assistant (CMA),0,1,0,NaN,NaN,NaN,NaN,0
3,Account Executive - Washington DC,0,1,0,Full-time,Mid-Senior level,Computer Software,Sales,0
4,Bill Review Manager,0,1,1,Full-time,Mid-Senior level,Hospital & Health Care,Health Care Provider,0


In [ ]:
#creating 2 columns country and state from location 
county_col = df["location"].str.split(pat = ",")
county_col.dropna()
country = []
state = []
for var in county_col:
  if var != None:
    if len(var) <= 3:
      if len(var) > 0:
        country.append(var[0])
      else:
        country.append(None)
      if len(var) > 1:
        state.append(var[1])
      else:
        state.append(None)
    else:
      country.append(None)
      state.append(None)

df_dropped["country"] = country
df_dropped["state"] = state
  
dataset = df_dropped[df_dropped['state'].notna()]

In [ ]:
#random forest model function 
def RandomForest(dataset):
  x_data = dataset.copy(deep = True)
  y_data = x_data['fraudulent']
  x_data.drop('fraudulent', axis=1, inplace = True)
  
  x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.15, random_state = 0)
  
  classifier = RandomForestClassifier(n_estimators= 10, criterion="entropy")
  
  classifier.fit(x_train, y_train)
  
  y_predr = classifier.predict(x_test)
  print(confusion_matrix(y_test, y_predr))

  print("accuracy score ->", round(accuracy_score(y_test, y_predr)*100, 2),"%")
  print("recall score ->", round(recall_score(y_test, y_predr)*100, 2), "%")
  print("precision score ->",round(precision_score(y_test, y_predr)*100, 2), "%")
  print("f1_score ->",round(f1_score(y_test,y_predr)*100, 2), "%")

  from sklearn.metrics import roc_auc_score
  print("auc score ->" , round(roc_auc_score(y_test, y_predr)*100, 2), "%")

In [ ]:
dataset.head()

,telecommuting,has_company_logo,has_questions,fraudulent,country,state
0,0,1,0,0,US,NY
1,0,1,0,0,NZ,
2,0,1,0,0,US,IA
3,0,1,0,0,US,DC
4,0,1,1,0,US,FL


In [ ]:
columnsToEncode = list(dataset.select_dtypes(include=['category','object']))
le = LabelEncoder()
for feature in columnsToEncode:
    
    try:
        dataset[feature] = le.fit_transform(dataset[feature])
        #print(feature)
    except:
        print('Error encoding '+feature)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
RandomForest(dataset)

[[2468   11]
 [ 108   11]]
accuracy score -> 95.42 %
recall score -> 9.24 %
precision score -> 50.0 %
f1_score -> 15.6 %
auc score -> 54.4 %
